In [1]:
local torch = require 'torch'
require 'hdf5'
require 'image'
require 'utilities'
require 'Rect' 
require 'lfs'
require 'LuaXML'

ASL_BASE_DIR = '/media/wei/DATA/datasets/vlm/snaps/'
ASL_ANNO_DIR = '/media/wei/DATA/datasets/vlm/annotations/'

img_size = {}
img_size['H'] = 58
img_size['W'] = 58

ground_truth = {}

class_names = {}
class_names[1] = 'Hand'
class_index = {}
class_index['Hand'] = 1

In [2]:
stored = load_obj(ASL_ANNO_DIR..'ASL_det_final.t7')

In [3]:
local ind = {}
for i =1, #stored.validation_set do
    ind[#ind+1] = i
end
shuffle(ind)

gb_ind = {}
for i = 1, math.floor(#stored.validation_set) do
    gb_ind[#gb_ind+1] = ind[i]
end
print(#gb_ind)

26827	


In [4]:
local ind = {}
for i =1, #stored.training_set do
    ind[#ind+1] = i
end
shuffle(ind)

tb_lb_ind = {}
for i = 1, math.floor(#stored.training_set) do
    tb_lb_ind[#tb_lb_ind+1] = ind[i]
end
print(#tb_lb_ind)

74618	


In [5]:
--Take 10,000 images totally from lb+tb+gb
-- 6,000 from gb, 4,000 from lb+tb
-- Split into train and test with ratio 0.8:0.2
-- i.e., trainset: 4800 from gb, 3200 from lb+tb
--       testset: 1200 from gb, 800 from lb+tb

trainset = {}
valset = {}
for i = 1, 6000 do
    if i <= 4800 then
        trainset[#trainset+1] = stored.validation_set[gb_ind[i]]
    else
        valset[#valset+1] = stored.validation_set[gb_ind[i]]
    end
end

for i = 1, 4000 do
    if i <= 3200 then
        trainset[#trainset+1] = stored.training_set[tb_lb_ind[i]]
    else
        valset[#valset+1] = stored.training_set[tb_lb_ind[i]]
    end 
end

print(#trainset)
print(#valset)

8000	
2000	


In [6]:
-- Shuffle trainset and testset again
print(trainset[1])
shuffle_n(trainset, #trainset)
print(trainset[1])

print(valset[1])
shuffle_n(valset, #valset)
print(valset[1])

/media/wei/DATA/datasets/vlm/snaps/gb1113/l/looks_5hs/0061.jpg	
/media/wei/DATA/datasets/vlm/snaps/tb1113/ASL_2008_05_12a/scene8-camera1/0370.jpg	
/media/wei/DATA/datasets/vlm/snaps/gb1113/i/inform_all/0035.jpg	
/media/wei/DATA/datasets/vlm/snaps/tb1113/ASL_2008_05_12a/scene44-camera1/2949.jpg	


In [7]:
save_obj(
    ASL_ANNO_DIR..'ASL_det_mix.t7',
    {
      dataset_name = stored.dataset_name,
      ground_truth = stored.ground_truth,
      training_set = trainset,
      validation_set = valset,
      class_names = stored.class_names,
      class_index = stored.class_index,
      background_files = stored.background_files
    }
)

In [9]:
t1 = 'test/'
tmp = string.format((t1..'eval%d.txt'), 10)
print(tmp)

test/eval10.txt	
